## Yolov11 Training Pipeline

---

#### Key Sections

1. [Training Setup](#training-setup)
2. [Training](#training)
3. [Hyperparameter Tuning](#hyperparameter-tuning)

---

### Training Setup

##### Imports

In [ ]:
import os
import gc
import re
import cv2
import random
import shutil
import torch
import warnings
import torchvision
from datetime import datetime
from ultralytics import YOLO
import matplotlib.pyplot as plt

##### Package Versions

In [2]:
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Is torch available?",torch.cuda.is_available())

Torch version: 2.7.0.dev20250127+rocm6.3
Torchvision version: 0.22.0.dev20250128+rocm6.3
Is torch available? True


### Disable Warnings

In [3]:
warnings.filterwarnings("ignore")

### Setting Training Device

In [4]:
# Defining Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Printing Necessary CUDA Info
if str(device) == "cuda":
    print(f"Using GPU {torch.cuda.get_device_properties(device)}")
    print("# of CUDA devices available:", torch.cuda.device_count())
else:
    print("Using CPU")

Using GPU _CudaDeviceProperties(name='AMD Radeon RX 7800 XT', major=11, minor=0, gcnArchName='gfx1101', total_memory=16368MB, multi_processor_count=30, uuid=38336232-6265-3432-3662-306564613532, L2_cache_size=4MB)
# of CUDA devices available: 1


### Emptying the GPU Cache (if necessary)

In [5]:
# Cleaning out the device cache
def empty_cache() -> None:
    gc.collect()
    torch.cuda.empty_cache()

# Displaying the free memory
def print_free_memory():
    free, total = torch.cuda.mem_get_info(device)
    print(f"Percent of free memory: {round(free/total *100,2)}")

# Running GPU info related functions
empty_cache()
print_free_memory()

Percent of free memory: 78.49


### Memory Summary

In [6]:
# Memory Summary Function
def memory_summary() -> None:
    print(torch.cuda.memory_summary())

# Running memory summary funciton
memory_summary()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

### Preparing GPU (if necessary)


In [7]:
# For AMD GPU - 7800xt
device_name = torch.cuda.get_device_name(0)
if "AMD" in device_name or "Radeon" in device_name:
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

print(f"GPU {torch.cuda.get_device_properties(device).name} is now setup")

GPU AMD Radeon RX 7800 XT is now setup


##### Local Setting Paths for Data, Base Model, and Output Directory


In [ ]:
# Directory Paths
training_dataset_name = "coco8" # Change this once we have the necessary data
yaml_training_dataset_name = "coco8.yaml" # Change this once we have the necessary data

# Flexible Data Paths (needed for THI server)
current_directory = os.getcwd()
path_to_base_directory = re.search(rf"(.*?){"Weird-Stuff-In-Traffic"}", current_directory).group(1)
training_yaml_data_path = f"Weird-Stuff-In-Traffic/Data/yolo/{training_dataset_name}/{yaml_training_dataset_name}"
complete_training_data_path = path_to_base_directory + training_yaml_data_path

# Model Paths
model_name = "yolo11n.pt"
simple_model_name = model_name.split(".")[0]

---

### Training

##### Training - Loading Model

In [ ]:
# Loading Model
model = YOLO(model_name)

##### Training - Training Configuration and Output Paths

In [ ]:
# Model Training Configurations
epochs = 3
image_size = 640
batch_size = 64

# Output Folder Name
output_folder_name = f'{datetime.now().strftime("%Y-%m-%d_%H-%M")}_{simple_model_name}_{training_dataset_name}_{image_size}cuts_{epochs}epoch'

# Model Storage and Results Path
training_results_path = f"Weird-Stuff-In-Traffic/Models/Segmentation-Detection/yolo/"
complete_training_results_path = path_to_base_directory + training_results_path + output_folder_name

##### Training - Training Parameters

In [12]:
# Model Training Parameters
training_params = {
    'data': complete_training_data_path, # Local Dataset
    'imgsz': image_size, #1024
    'epochs': epochs,
    'batch': batch_size,
    'patience': 20,
    'cos_lr': True,
    #'rect': True,
    'augment': True,
    #'hsv_s': 0.45,
    #'hsv_v': 0.3,
    #'auto_augment': 'autoaugment',
    'save': True,
    'project': complete_training_results_path, # Local Save Directory
    'name':  output_folder_name # Output Folder Name
}

##### Training - Training Model

In [14]:
results = model.train(**training_params)

New https://pypi.org/project/ultralytics/8.3.114 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.103 🚀 Python-3.12.3 torch-2.7.0.dev20250127+rocm6.3 CUDA:0 (AMD Radeon RX 7800 XT, 16368MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/test_set_chunk3/chunk3.yaml, epochs=1, time=None, patience=20, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch, name=2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val

train: Scanning /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/test_set_chunk3/labels/train... 142 images, 2 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<00:00, 4954.63it/s]

train: New cache created: /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/test_set_chunk3/labels/train.cache



val: Scanning /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/test_set_chunk3/labels/val... 32 images, 1 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<00:00, 2554.29it/s]

val: New cache created: /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/test_set_chunk3/labels/val.cache


Plotting labels to /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1     0.254G      1.829      4.367      1.143         35        640: 100%|██████████| 3/3 [02:17<00:00, 45.71s/it] 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:10<00:00, 10.41s/it]


                   all         33         37    0.00131      0.351     0.0027   0.000918

1 epochs completed in 0.041 hours.
Optimizer stripped from /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2/weights/last.pt, 5.5MB
Optimizer stripped from /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2/weights/best.pt, 5.5MB

Validating /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2/weights/best.pt...
Ultralytics 8.3.103 🚀 Python-3.12.3 torch-2.7.0.dev20250127+rocm6.3 CUDA:0 (AMD Radeon RX 7800 XT, 16368MiB)
YOLO11n summary (fus

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:13<00:00, 13.27s/it]


                   all         33         37    0.00141      0.378    0.00157   0.000528
Speed: 0.1ms preprocess, 399.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Model/Segmentation-Detection/yolo/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch/2025-04-23_13-11_yolo11n_test_set_chunk3_640cuts_1epoch2


##### Training - Printing Key Training Metrics

In [ ]:
# After training, print the most important metrics
print("\n### Training completed. ### \n\n Key metrics:")
print(f"X (all classes): {results.box.map:.4f}")  #??
print(f"mAP50 (all classes): {results.box.map50:.4f}")
print(f"mAP50-95 (all classes)??: {results.box.map75:.4f}")  # This might be mAP50-95??
#print(f"Precision: {results.box.p:.4f}")
#print(f"Recall: {results.box.r:.4f}")

# Print class-specific metrics if available
if hasattr(results, 'names'):
    print("\nClass-specific metrics:")
    for i, class_name in results.names.items():
        print(f"{class_name}:")
        print(f"  Precision: {results.box.p[i]:.4f}")
        print(f"  Recall: {results.box.r[i]:.4f}") 
        #print(f"  mAP50: {results.box.map50[i]:.4f}") # This breaks the code, revisit later
        #print(f"  mAP50-95: {results.box.map[i]:.4f}") # This breaks the code, revisit later


### Training completed. ### 

 Key metrics:
X (all classes): 0.0005
mAP50 (all classes): 0.0016
mAP50-95 (all classes)??: 0.0003

Class-specific metrics:
weird:
  Precision: 0.0014
  Recall: 0.3784

Training complete!


---

### Hyperparameter Tuning

##### Builtin ray tune (Hyperband)

In [ ]:
'''
from ultralytics import YOLO
from ray import tune

model_name = "yolo11n.pt"
model = YOLO(model_name)
hypp_space = {}

result_grid = model.tune(
    device='cuda:0',
    data=dataset_path,
    iterations=3,
    grace_period = 2,
    name="tune_exp_1", 
    #space = hypp_space,
    #resume=True,
    imgsz=1024,
    epochs=2,
    batch=24,
    project=save_dir,
    use_ray=True)
'''


(_tune pid=26916) New https://pypi.org/project/ultralytics/8.3.112 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=26916) Ultralytics 8.3.111 🚀 Python-3.12.10 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
(_tune pid=26916) engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\data.yaml, epochs=2, time=None, patience=100, batch=24, imgsz=1024, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=C:\Users\Adminero\Documents\Valomalo\Saved Runs, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source

  0%|          | 0.00/5.35M [00:00<?, ?B/s]
  7%|▋         | 384k/5.35M [00:00<00:01, 3.10MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 5.14MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 4.95MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 3.18MB/s]


(_tune pid=26916) AMP: checks passed ✅


train: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\train\labels.cache... 2471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2471/2471 [00:00<?, ?it/s]


(_tune pid=26916) train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 385.3±55.1 MB/s, size: 173.3 KB)


  0%|          | 0.00/5.35M [00:00<?, ?B/s] [repeated 2x across cluster]
 84%|████████▍ | 4.50M/5.35M [00:01<00:00, 4.48MB/s] [repeated 27x across cluster]
val: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\val\labels.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s] [repeated 3x across cluster]


(_tune pid=27636) Overriding model.yaml nc=80 with nc=2 [repeated 2x across cluster]
(_tune pid=27636)                    from  n    params  module                                       arguments                      [repeated 2x across cluster]
(_tune pid=27636)  20                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 14x across cluster]
(_tune pid=27636)   4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]      [repeated 4x across cluster]
(_tune pid=27636)  22                  -1  1    378880  ultralytics.nn.modules.block.C3k2            [384, 256, 1, True]            [repeated 6x across cluster]
(_tune pid=27636)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 2x across cluster]
(_tune pid=27636)  14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2

2025-04-21 17:59:31,882	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-04-21 17:59:31,886	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Adminero/Documents/Valomalo/Saved Runs/tune_exp_18' in 0.0033s.
  0%|          | 0/103 [00:00<?, ?it/s]
val: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\val\labels.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s] [repeated 2x across cluster]


(_tune pid=27636) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.016354728253908123' and 'momentum=0.7360041068128451' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=27636) optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0008806517954492481), 87 bias(decay=0.0)
(_tune pid=27636) Image sizes 1024 train, 1024 val
(_tune pid=27636) Using 8 dataloader workers
(_tune pid=27636) Logging results to C:\Users\Adminero\Documents\Valomalo\Saved Runs\train13
(_tune pid=27636) Starting training for 2 epochs...
(_tune pid=27636) 
(_tune pid=27636)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=27132) Plotting labels to C:\Users\Adminero\Documents\Valomalo\Saved Runs\train14\labels.jpg...  [repeated 2x across cluster]
(_tune pid=27132) 


##### Builtin YOLO tune (genetic algorithm)

In [ ]:
'''
from ultralytics import YOLO

model_name = "yolo11n.pt"
model = YOLO(model_name)
hypp_space = {}

model.tune(
    device='cuda:0',
    data=dataset_path,
    epochs=30,
    name="tune_exp_1", 
    imgsz=1024,
    batch=24,
    iterations=300,
    optimizer="AdamW",
    plots=True,
    save=False,
    val=False,
)
'''

##### Custom Optuna trial (Bayesian optimization)

In [ ]:
'''
import optuna
from ultralytics import YOLO

def objective(trial):
    # Define hyperparameters to optimize
    lr = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.0001, 0.01)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    
    # Initialize YOLO model
    model = YOLO('yolov8n.yaml')  
    
    # Train with suggested hyperparameters
    results = model.train(
        data='coco128.yaml',
        epochs=50,
        batch=batch_size,
        lr0=lr,
        weight_decay=weight_decay,
        dropout=dropout,
        verbose=False  # Reduce output clutter
    )
    
    # Return the metric to optimize 
    return results.results_dict['metrics/mAP50-95(B)']

# Create study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print best results
print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')
'''